In [11]:
import sys
import os
sys.path.append(os.path.abspath('../..')) # include top level package in python path

In [12]:
import torch
from comparison.examples.vae_toy import VAE_Toy
from comparison.loss import ELBO, IWAE
from tqdm.notebook import tqdm

In [13]:
# Change device as needed. 
# CPU is supported on all machines, CUDA for specific GPUs.

# device = 'cpu'
device = 'cuda'

In [14]:
def train(model, optim, dataloader, no_epochs=3):
    model.train()
    
    train_loss = []
    for epoch in range(1, no_epochs + 1):
        for imgs, _ in tqdm(dataloader, desc="Training Epoch №%s" % epoch, leave=False):
            imgs = imgs.view(-1, 28*28).to(device)
            vae_res = model(imgs, K=8, M=8)
            loss = -ELBO(vae_res)
            train_loss.append(loss.item())

            optim.zero_grad()
            loss.backward()
            optim.step()

    return torch.Tensor(train_loss)

In [18]:
model = VAE_Toy(D=3).to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)